In [47]:
# Data Manipulation
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Analysis
from sklearn.metrics import mean_absolute_error
import scipy as sp
import statsmodels as sm
from scipy.stats import wilcoxon, mannwhitneyu

In [2]:
df_tf_input = pd.read_csv("data-collection/outputs/performance-data-true-false.csv")
df_ft_input = pd.read_csv("data-collection/outputs/performance-data-false-true.csv")
df_ff_input = pd.read_csv("data-collection/outputs/performance-data-false-false.csv")

df_gt_input = pd.read_csv("data-collection/groundtruths/target-data.csv")

### Compare number of corners found

In [3]:
# Add column to each DF saying how many corners are not Nan
df_tf = df_tf_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_ff = df_ff_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_gt = df_gt_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]

df_tf = df_tf.replace('FAILED', np.nan, regex=True)
df_ff = df_ff.replace('FAILED', np.nan, regex=True)

df_tf["tf-num-corners"] = df_tf.count(axis=1)-1
df_ff["ff-num-corners"] = df_ff.count(axis=1)-1
df_gt["gt-num-corners"] = df_gt.count(axis=1)-1

c:\Users\Kim\anaconda3\envs\analysis\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [4]:
# Remove all other columns but name and corner num
df_tf = df_tf[["filename", "tf-num-corners"]]
df_ff = df_ff[["filename", "ff-num-corners"]]
df_gt = df_gt[["filename", "gt-num-corners"]]

In [5]:
# Merge on filename
corner_nums = df_gt.merge(df_tf, on="filename").merge(df_ff, on="filename")

In [6]:
# Get MAE compared to GTs
error_tf = mean_absolute_error(corner_nums["gt-num-corners"], corner_nums["tf-num-corners"])
error_ff = mean_absolute_error(corner_nums["gt-num-corners"], corner_nums["ff-num-corners"])

In [7]:
print(error_tf)
print(error_ff)

0.8309859154929577
1.1549295774647887


In [8]:
tf_error_dist = np.abs(corner_nums["gt-num-corners"]-corner_nums["tf-num-corners"])
ff_error_dist = np.abs(corner_nums["gt-num-corners"]-corner_nums["ff-num-corners"])

res = wilcoxon(tf_error_dist, ff_error_dist)

print(res)

WilcoxonResult(statistic=48.0, pvalue=0.0002451354174095733)


### Compare error of corner points

In [74]:
df_tf = df_tf_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_ff = df_ff_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]
df_gt = df_gt_input[["filename", "corner-1", "corner-2", "corner-3", "corner-4", "corner-5"]]

df_tf = df_tf.replace('FAILED', np.nan, regex=True)
df_ff = df_ff.replace('FAILED', np.nan, regex=True)

In [ ]:
# Compare each corner point to each ground truth corner point and save the error of whichever is closest

### Error of corner-floor intersection points

In [9]:
df_ft = df_ft_input[["filename", "co-fl-in-1", "co-fl-in-2", "co-fl-in-3", "co-fl-in-4", "co-fl-in-5"]]
df_gt = df_gt_input[["filename", "co-fl-in-1", "co-fl-in-2", "co-fl-in-3", "co-fl-in-4", "co-fl-in-5"]]

df_ft = df_ft.replace('FAILED', np.nan, regex=True)

In [ ]:
# calculate error

# 30 pixels

### Compare wall coverage percentage

In [66]:
df_tf = df_tf_input[["filename", "wall-coverage-perc"]]
df_ft = df_ft_input[["filename", "wall-coverage-perc"]]
df_ff = df_ff_input[["filename", "wall-coverage-perc"]]

df_tf = df_tf.replace('FAILED', 0, regex=True)
df_ft = df_ft.replace('FAILED', 0, regex=True)
df_ff = df_ff.replace('FAILED', 0, regex=True)

In [67]:
df_tf["wall-coverage-perc"] = df_tf["wall-coverage-perc"].astype(float)
df_ft["wall-coverage-perc"] = df_ft["wall-coverage-perc"].astype(float)
df_ff["wall-coverage-perc"] = df_ff["wall-coverage-perc"].astype(float)

In [68]:
# discount those where plain wall mask is used by removing 1s
df_tf = df_tf.drop(df_tf[df_tf['wall-coverage-perc'] == float(1)].index)
df_ft = df_ft.drop(df_ft[df_ft['wall-coverage-perc'] == float(1)].index)
df_ff = df_ff.drop(df_ff[df_ff['wall-coverage-perc'] == float(1)].index)

In [69]:
tf_mean = df_tf[["wall-coverage-perc"]].mean(numeric_only=True)[0]
ft_mean = df_ft[["wall-coverage-perc"]].mean(numeric_only=True)[0]
ff_mean = df_ff[["wall-coverage-perc"]].mean(numeric_only=True)[0]

print(f"tf_mean = {tf_mean}")
print(f"ft_mean = {ft_mean}")
print(f"ff_mean = {ff_mean}")

tf_mean = 0.9137844023469459
ft_mean = 0.8236358285052049
ff_mean = 0.9166521378509982


In [73]:
res_1 = mannwhitneyu(df_tf["wall-coverage-perc"], df_ft["wall-coverage-perc"])
res_2 = mannwhitneyu(df_tf["wall-coverage-perc"], df_ff["wall-coverage-perc"])
res_3 = mannwhitneyu(df_ft["wall-coverage-perc"], df_ff["wall-coverage-perc"])

print(f"res_1 = {res_1}")
print(f"res_2 = {res_2}")
print(f"res_3 = {res_3}")

res_1 = MannwhitneyuResult(statistic=1854.5, pvalue=0.4767893960305616)
res_2 = MannwhitneyuResult(statistic=1550.0, pvalue=0.5517142360306657)
res_3 = MannwhitneyuResult(statistic=1173.5, pvalue=0.248407364344584)
